# RQ1: Community Opinion Experiment

This notebook runs the main RQ1 experiment with Retriever, Summarizer, and Agent modules.
Experiments: Ours, RAG, RANDOM, CONV, COMM

## Setup and Configuration

In [ ]:
import os
import pandas as pd
import numpy as np
import re
import time
import glob
from datetime import datetime
from tqdm import tqdm
import torch
import faiss
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from openai import OpenAI
import anthropic

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "YOUR_GOOGLE_API_KEY_HERE")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_API_KEY_HERE")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "YOUR_ANTHROPIC_API_KEY_HERE")

genai.configure(api_key=GOOGLE_API_KEY)

TOPIC_KEYWORDS = [
    "정권 교체", "통합 정치", "단일화(윤석열-안철수)",
    "부동산, 세금 등 경제문제", "여성가족부 폐지",
    "후보(또는 가족)의 비리", "대장동 의혹"
]

COMMUNITY_MAPPING = {
    "fm": "에펨코리아",
    "mlb": "MLBPARK",
    "pp": "뽐뿌"
}

In [ ]:
def call_llm(prompt, model_type="gemini", **kwargs):
    if model_type == "gemini":
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        return response.text
    elif model_type == "gpt":
        client = OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            **kwargs
        )
        return response.choices[0].message.content
    elif model_type == "claude":
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        message = client.messages.create(
            model="claude-3-5-haiku-20241022",
            max_tokens=kwargs.get("max_tokens", 4096),
            messages=[{"role": "user", "content": prompt}]
        )
        return message.content[0].text
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

## Load Data and FAISS Index

In [ ]:
torch.cuda.empty_cache()

community = "fm"

df_sample = pd.read_csv(f"../dataset/community_samples/{community}_sample_50000.csv", encoding="utf-8-sig")
embedding_model = SentenceTransformer("intfloat/multilingual-e5-small", device="cuda")

index_path = f"../dataset/faiss/{community}_embeddings_faiss_50000.index"
if os.path.exists(index_path):
    index = faiss.read_index(index_path)
    print(f"Loaded FAISS index: {index.ntotal} vectors")
else:
    raise FileNotFoundError(f"FAISS index not found: {index_path}")

In [ ]:
def chunk_text(text, max_chunk_length=512):
    words = text.split()
    chunks, current = [], []
    for word in words:
        if len(" ".join(current + [word])) <= max_chunk_length:
            current.append(word)
        else:
            chunks.append(" ".join(current))
            current = [word]
    if current:
        chunks.append(" ".join(current))
    return chunks

text_chunks_unique = []
seen_chunks = set()

for _, row in df_sample.iterrows():
    for chunk in chunk_text(str(row["Text"])):
        if chunk not in seen_chunks:
            seen_chunks.add(chunk)
            chunked_row = row.to_dict()
            chunked_row["Text"] = chunk
            text_chunks_unique.append(chunked_row)

chunked_df = pd.DataFrame(text_chunks_unique)
chunked_df["ProcessedText"] = chunked_df["Text"].apply(lambda x: f"query: {x}")
print(f"Chunked DataFrame: {len(chunked_df)} rows")

## Retriever Module (augment_text)

In [ ]:
AUGMENT_PROMPT = """
Please write exactly two lines in Korean about the topic '{topic}':
1. The first line should briefly state the reason for supporting.
2. The second line should briefly state the reason for opposing.
"""

def augment_text(topic, top_k, question, model_type="gemini"):
    prompt = AUGMENT_PROMPT.format(topic=topic)
    aug_response = call_llm(prompt, model_type=model_type)
    
    query_emb = embedding_model.encode([f"query: {aug_response}"], convert_to_numpy=True, normalize_embeddings=True).astype("float32")
    distances, indices = index.search(query_emb, top_k)
    
    retrieved_texts = chunked_df.iloc[indices[0]]["Text"].tolist()
    return aug_response, retrieved_texts

## Summarizer Module (extract_opinions)

In [ ]:
SUMMARIZER_PROMPT = """
You are a sophisticated qualitative analysis AI.
Your primary function is to extract and analyze multiple dominant themes from social media posts.
Your goal is to identify the various topics, sentiments, and arguments that emerge in public discussions.

Core Task: Extract Multiple Dominant Themes
Answer the question: "{question}" based on the provided posts.
Rather than finding a single 'center', identify ALL significant themes that appear in the conversation.

Analysis Workflow:
1. Initial Scanning: Read all posts to identify distinct themes and topics being discussed.
2. Theme Extraction: Identify 2-4 major themes based on frequency, engagement, and significance.

Theme Identification Guidelines:
- Each theme should be distinct and substantive enough to stand alone
- Themes can be completely unrelated to each other
- Include both majority and significant minority themes
- For each theme, accurately represent its dominant sentiment without artificial balancing

Posts:
{opinions}

Output Format (Korean):
<answer>
1. [Theme]: [Key points and sentiment summary]
2. [Theme]: [Key points and sentiment summary]
3. [Theme]: [Key points and sentiment summary]
</answer>
"""

def extract_opinions(question, texts, model_type="gemini"):
    opinions = "\n---\n".join(texts)
    prompt = SUMMARIZER_PROMPT.format(question=question, opinions=opinions)
    return call_llm(prompt, model_type=model_type)

## Agent Module (final_agent)

In [ ]:
FINAL_AGENT_PROMPT = """
Please select your major one opinion below based on the overall opinion. Do nothing else.
{persona}

<question>
{question}
"""

def final_agent(analysis, question, model_type="gemini"):
    prompt = FINAL_AGENT_PROMPT.format(persona=analysis, question=question)
    return call_llm(prompt, model_type=model_type)

## Baseline Agents (RAG, Random, Conv, Comm)

In [ ]:
RAG_PROMPT = """
Please select your major opinion below based on the overall opinion. Do nothing else.
{persona}

<question>
{question}
"""

def rag_agent(question, opinions_str, model_type="gemini"):
    prompt = RAG_PROMPT.format(persona=opinions_str, question=question)
    return call_llm(prompt, model_type=model_type)

def random_agent(question, model_type="gemini"):
    random_opinions = "\n---\n".join(chunked_df.sample(n=50)["Text"].values)
    prompt = RAG_PROMPT.format(persona=random_opinions, question=question)
    return call_llm(prompt, model_type=model_type), random_opinions

COMM_PROMPT = """
You are a member of {community} in South Korea.
Please select your opinion based on this community's perspective. Do nothing else.

<question>
{question}
"""

def comm_agent(question, community_name, model_type="gemini"):
    prompt = COMM_PROMPT.format(community=community_name, question=question)
    return call_llm(prompt, model_type=model_type)

CONV_PROMPT = """
You are a member of conservative group of South Korea.
Please select your opinion based on this community's perspective. Do nothing else.

<question>
{question}
"""

def conv_agent(question, model_type="gemini"):
    prompt = CONV_PROMPT.format(question=question)
    return call_llm(prompt, model_type=model_type)

## Accuracy Calculation

In [ ]:
def parse_answer_letter(raw_answer_str):
    s = str(raw_answer_str).replace("\u00A0", " ").strip()
    s = re.sub(r"['\"\u2018\u2019\u201c\u201d`]", "", s)
    
    match = re.match(r"^([A-D])", s, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    
    match = re.search(r"([A-D])[.)]", s, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    
    match = re.search(r"\b([A-D])\b", s, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    
    return None

def calculate_accuracy(predictions_df, ground_truth_df, community="fm"):
    parsed = predictions_df["response"].apply(parse_answer_letter)
    correct = (parsed == ground_truth_df[community]).sum()
    total = len(ground_truth_df)
    return correct / total if total > 0 else 0.0

## Run Experiments

In [ ]:
questions_df = pd.read_csv("../dataset/RQ1_questions/selected_questions/call_experiment_survey.csv")
top_k = 50
num_iterations = 100

for model_type in ["gemini", "gpt", "claude"]:
    print(f"\n=== Running experiments with {model_type} ===")
    
    for topic in TOPIC_KEYWORDS:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        results = []
        
        topic_questions = questions_df[questions_df["topic"] == topic] if "topic" in questions_df.columns else questions_df
        
        for idx, row in tqdm(topic_questions.iterrows(), desc=f"{topic}"):
            question = row["question"]
            
            for seed in range(num_iterations):
                aug_text, retrieved = augment_text(topic, top_k, question, model_type=model_type)
                summary = extract_opinions(question, retrieved, model_type=model_type)
                response = final_agent(summary, question, model_type=model_type)
                
                results.append({
                    "question": question,
                    "seed": seed,
                    "response": response,
                    "method": "ours"
                })
                time.sleep(2)
        
        output_dir = f"../results/RQ1/{community}"
        os.makedirs(output_dir, exist_ok=True)
        pd.DataFrame(results).to_csv(
            f"{output_dir}/{community}_ours_{model_type}_{topic}_{timestamp}.csv",
            index=False, encoding="utf-8-sig"
        )

## Analyze Results

In [ ]:
result_files = glob.glob(f"../results/RQ1/{community}/{community}_ours_*.csv")
if result_files:
    all_results = pd.concat([pd.read_csv(f, encoding="utf-8-sig") for f in result_files])
    
    ground_truth = pd.read_csv("../dataset/RQ1_questions/selected_questions/call_experiment_survey.csv")
    accuracy = calculate_accuracy(all_results, ground_truth, community=community)
    print(f"Overall accuracy for {community}: {accuracy:.4f}")